In [1]:
import json
import os
import openai

with open('keys.json', "r") as f:
    keys = json.load(f)
print(keys.keys())

openai.api_key = keys['secret']

dataDir = "datasets/"
with open(dataDir + "counterfact.json", "r") as f:
    counterfact = json.load(f)
if os.path.exists(dataDir + "related.json"):
    with open(dataDir +"related.json", "r") as f:
        related = json.load(f)
else:
    related = []
print(len(counterfact))
print(len(related))
totalTokens = 0

def costEstimate(tokens):
    return (tokens/1000) * 0.002


dict_keys(['secret'])
21919
21919


In [2]:
def genMsg(idx):
    msg = counterfact[idx]['requested_rewrite']['prompt']
    msg = msg.replace("{}", counterfact[idx]['requested_rewrite']['subject'])
    msg = msg + " " + counterfact[idx]['requested_rewrite']['target_true']["str"] + "."
    msg = msg + f" Can you write a list on one line without commas of related words to the subject of the first sentence, {counterfact[idx]['requested_rewrite']['subject']}."
    return msg

In [3]:
def genResponse(msg):
    global totalTokens
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": msg}
        ],
        max_tokens=20,
        # stop=["\n", "."],
        temperature=0.8,
        presence_penalty=0.0,
    )
    totalTokens += response["usage"]["total_tokens"]
    return response

In [4]:
punctuation = [".", ",", "!", "?", ";", ":", "-", "(", ")", "[", "]", "{", "}", "<", ">"]
def postProcess(words):
    for punc in punctuation:
        words = words.replace(punc, " ")
    words = words.split(" ")
    words = [word for word in words if word != ""]
    return words

In [5]:
i = len(related)
print(counterfact[i])
msg = genMsg(i)
print(msg)

IndexError: list index out of range

In [ ]:
# response = genResponse(msg)

In [ ]:
# print(response["choices"][0]["message"]["content"])
# print(postProcess(response["choices"][0]["message"]["content"]))
# print(response["choices"][0]["finish_reason"])
# print(response["usage"]["total_tokens"])
# print(costEstimate(totalTokens))

In [8]:
testing = False

for i in range(len(related), len(counterfact)):
    msg = genMsg(i)
    response = genResponse(msg)
    words = postProcess(response["choices"][0]["message"]["content"])
    if testing:
        print(msg)
        print(response["choices"][0]["message"]["content"])
        print(words)

    entry = counterfact[i]
    entry["relatedWords"] = words
    entry["responseRaw"] = response.to_dict()
    related.append(entry)
    with open(dataDir + "related.json", "w") as f:
        json.dump(related, f)

    # print(response["choices"][0]["finish_reason"])
    print(f"{i} / {len(counterfact)} Current Cost {costEstimate(totalTokens)}", end="\r")
    if costEstimate(totalTokens) > 10:
        break

    if testing:
        break

In [6]:
print(related[-1].keys())
print(related[-1]["relatedWords"])
print(len(related))

dict_keys(['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts', 'relatedWords', 'responseRaw'])
['actor', 'theatre', 'film', 'stage', 'performance', 'drama', 'comedy', 'tragedy', 'audition', 'rehearsal', 'script', 'character', 'role', 'director', 'producer', 'cinematographer', 'screenplay', 'set', 'design']
21919


In [28]:
from collections import deque
import numpy as np
from torch.utils.data.dataset import IterableDataset, Dataset
from torch.utils.data import DataLoader
import torch
from torchinfo import summary
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoConfig

from core import padded_stack, stack_dicts_list

In [43]:
with open(dataDir +"related.json", "r") as f:
    related = json.load(f)

In [57]:
# json with strings to tuple of tensors
class RelatedDataset(Dataset):
    def __init__(self, dataJson, tokenizer):
        self.dataJson = dataJson
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataJson)

    def __getitem__(self, index):
        prompt, response, related = self.formatText(index)

        promptT = self.tokenizer(prompt, return_tensors="pt")["input_ids"].squeeze(0)
        responseT = self.tokenizer(response, return_tensors="pt")["input_ids"].squeeze(0)
        relatedT = self.tokenizer(related, return_tensors="pt")["input_ids"].squeeze(0)

        return promptT, responseT, relatedT

    def formatText(self, idx):
        prompt = self.dataJson[idx]['requested_rewrite']['prompt']
        prompt = prompt.replace("{}", self.dataJson[idx]['requested_rewrite']['subject'])
        response = counterfact[idx]['requested_rewrite']['target_true']["str"] + "."
        related = self.dataJson[idx]['relatedWords']
        relatedStr = related[0]
        for r in related[1:]:
            relatedStr += " " + r
        return prompt, response, relatedStr
    
# batching and padding, use left pad tokkenizer
class RelatedCollator():
    def __init__(self, tokenizer):
        self.textCollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
        self.pad_token_id = tokenizer.pad_token_id

    def __call__(self, batch):
        prompt, response, related = zip(*batch)

        print(prompt)
        prompt = self.textCollator(prompt)
        response = self.textCollator(response)
        related = self.textCollator(related)

        prompt["attention_mask"] = torch.where(prompt["input_ids"] == self.pad_token_id, 0, 1)
        response["attention_mask"] = torch.where(response["input_ids"] == self.pad_token_id, 0, 1)
        related["attention_mask"] = torch.where(related["input_ids"] == self.pad_token_id, 0, 1)

        return prompt, response, related # token index in ["input_ids"], -100 for pad in ["labels"]

In [58]:
# left pad so all generations start at the same index
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left', return_special_tokens_mask=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(tokenizer.pad_token_id)
dataset = RelatedDataset(related, tokenizer)
collator = RelatedCollator(tokenizer)
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collator, shuffle=True)

50257


In [59]:
# test dataloader
for i, batch in enumerate(dataloader):
    print(batch[0].keys())
    print(batch[0]["input_ids"])
    print(batch[0]["labels"])
    print(batch[0]["attention_mask"])
    print(batch[1]["input_ids"])
    print(batch[1]["labels"])
    print(batch[1]["attention_mask"])
    print(batch[2]["input_ids"])
    print(batch[2]["labels"])
    print(batch[2]["attention_mask"])
    break

(tensor([7554, 1766, 1616,  373, 4642,  287]), tensor([3351,  295,  256,   34,   11, 4635,  416]))
dict_keys(['input_ids', 'labels', 'attention_mask'])
tensor([[50257,  7554,  1766,  1616,   373,  4642,   287],
        [ 3351,   295,   256,    34,    11,  4635,   416]])
tensor([[-100, 7554, 1766, 1616,  373, 4642,  287],
        [3351,  295,  256,   34,   11, 4635,  416]])
tensor([[0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])
tensor([[50257, 23421,    13],
        [48236,  4265,    13]])
tensor([[ -100, 23421,    13],
        [48236,  4265,    13]])
tensor([[0, 1, 1],
        [1, 1, 1]])
tensor([[50257, 50257, 50257, 23421,  4082,  3594, 37275,   666, 15606,  3707,
         29396,   520,  3362,   338,  3961],
        [48236,  4265, 15062,   431,  7092,  1097,  4930,  3420,  8880,  7825,
          3708, 31537,  1891, 17969, 11478]])
tensor([[ -100,  -100,  -100, 23421,  4082,  3594, 37275,   666, 15606,  3707,
         29396,   520,  3362,   338,  3961],
        [48236,  4265, 